#### Build & Publish

pip install spatial-transform --upgrade
python -m pipreqs.pipreqs --force

In [3]:
from SpatialTransform import Transform, Euler

# defining the transforms
hips = Transform('Hips', position=(0,2,0))
LeftLegUpper = Transform('LeftLegUpper', position=(+0.2,0,0))
LeftLegLower = Transform('LeftLegLower', position=(0,-1,0))
LeftLegFoot = Transform('LeftLegFoot', position=(0,-1,0))
RightLegUpper = Transform('RightLegUpper', position=(-0.2,0,0))
RightLegLower = Transform('RightLegLower', position=(0,-1,0))
RightLegFoot = Transform('RightLegFoot', position=(0,-1,0))

# defining the hierarchy
hips.attach(LeftLegUpper)
LeftLegUpper.attach(LeftLegLower)
LeftLegLower.attach(LeftLegFoot)

hips.attach(RightLegUpper)
RightLegUpper.attach(RightLegLower)
RightLegLower.attach(RightLegFoot)

# show the created hierarchy
hips.printTree()
print('\nWorld positions, local positions, joint directions:')
for item, index, depth in hips.layout():
    print(f'{item.PositionWorld} {item.Position} {item.ForwardWorld} {item.Name}')

Hips
+- LeftLegUpper
|  +- LeftLegLower
|     +- LeftLegFoot
+- RightLegUpper
   +- RightLegLower
      +- RightLegFoot

World positions, local positions, joint directions:
vec3(            0,            2,            0 ) vec3(            0,            2,            0 ) vec3(            0,            0,           -1 ) Hips
vec3(          0.2,            0,            0 ) vec3(          0.2,           -2,            0 ) vec3(            0,            0,           -1 ) LeftLegUpper
vec3(            0,           -1,            0 ) vec3(         -0.2,           -1,            0 ) vec3(            0,            0,           -1 ) LeftLegLower
vec3(            0,           -1,            0 ) vec3(            0,            0,            0 ) vec3(            0,            0,           -1 ) LeftLegFoot
vec3(         -0.2,            0,            0 ) vec3(         -0.2,           -2,            0 ) vec3(            0,            0,           -1 ) RightLegUpper
vec3(            0,           -1,  

In [4]:
from SpatialTransform import Transform

# the basic properties of the transform as position, scale and rotation can be changed by setting the value
# but the inverse-properties are read only 
root = Transform()
root.PositionWorld = (1,2,3)
root.Scale = .1                     # accepts either a single value or a tuple of three
root.RotationWorld = (1, 0, 0, 0)   # rotations are in quaternions

# the rotation can be also read and changed with extra methods for simplified usage
root.setEuler((0, 90, 0))
root.getEuler(order='ZYX')
root.lookAtWorld((1, 1, 1))

# some methods do update the transform and keep childrens spatially unchanged
root.clearParent(keep=['position', 'rotation', 'scale'])
root.clearChildren(keep=['position', 'rotation', 'scale'])
root.applyPosition()
root.applyRotation(recursive=True)
root.applyScale(recursive=True)

# the transform provide two methods to convert arbitrary points and direction from and to the spaces
root.pointToWorld((5,4,3))
root.directionToLocal((2,3,4))


vec3( 2, 3, 4 )

In [5]:
from SpatialTransform import Transform

# because almost every method on the "Transform" object returns itself,
# the previous code of creating and attaching can also be written like:
hips = Transform('Hips', position=(0,2,0)).attach(
    Transform('LeftLegUpper', position=(+0.2,0,0)).attach(
        Transform('LeftLegLower', position=(0,-1,0)).attach(
            Transform('LeftLegFoot', position=(0,-1,0))
        )
    ),
    Transform('RightLegUpper', position=(-0.2,0,0)).attach(
        Transform('RightLegLower', position=(0,-1,0)).attach(
            Transform('RightLegFoot', position=(0,-1,0))
        )
    )
)

# multiple actions on a transform can be performed on a single line
feets = hips.setEuler((0, 180, 0)).applyRotation().filter('Foot')

# show the created hierarchy
hips.printTree()
print('\nPositions:')
for item, index, depth in hips.layout():
    print(f'{item.PositionWorld} {item.Position} {item.Name}')

Hips
+- LeftLegUpper
|  +- LeftLegLower
|     +- LeftLegFoot
+- RightLegUpper
   +- RightLegLower
      +- RightLegFoot

Positions:
vec3(            0,            2,            0 ) vec3(            0,            2,            0 ) Hips
vec3(         -0.2,            0,  1.74846e-08 ) vec3(         -0.2,           -2,  1.74846e-08 ) LeftLegUpper
vec3(            0,           -1,            0 ) vec3(         -0.2,           -1,            0 ) LeftLegLower
vec3(            0,           -1,            0 ) vec3(            0,            0,            0 ) LeftLegFoot
vec3(          0.2,            0, -1.74846e-08 ) vec3(          0.2,           -2, -1.74846e-08 ) RightLegUpper
vec3(            0,           -1,            0 ) vec3(          0.2,           -1,            0 ) RightLegLower
vec3(            0,           -1,            0 ) vec3(            0,            0,            0 ) RightLegFoot


In [6]:
# the package also provides the static class 'Euler'
# the 'Transform' does also rely on that to convert between rotation representations
from SpatialTransform import Euler

# rotations are in radians here
matrix = Euler.toMatFrom((1, 2, .5), order='YZX', extrinsic=True)
quaternion = Euler.toQuatFrom((1, 2, .5), order='YZX', extrinsic=True)

angles1 = Euler.fromMatTo(matrix, order='XYZ', extrinsic=False)
angles2 = Euler.fromQuatTo(quaternion, order='XYZ', extrinsic=False)

print(angles1 - angles2)

vec3(            0,            0,            0 )
